<a href="https://colab.research.google.com/github/GuptaNavdeep1983/CS688/blob/main/CS688_Assignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:

https://cran.r-project.org/web/packages/httr/vignettes/quickstart.html



In [ ]:
install.packages("hash")
install.packages("syuzhet")
install.packages(c("httr", "jsonlite"))
install.packages("")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘data.table’, ‘slam’, ‘textshape’, ‘NLP’, ‘zoo’, ‘dtt’


Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(dplyr) # for pipes and the data_frame function
library(rvest) # webscraping
library(stringr) # to deal with strings and to clean up our data
library(curl)
library(hash)
library(syuzhet)	
library(httr)
library(jsonlite)


r <- GET("https://api.cognitive.microsoft.com/bing/v7.0/news/search?q=category=Coronavirus&count=20", add_headers("Ocp-Apim-Subscription-Key" = "XXX"))


jsonRespParsed<-content(r,as="parsed")
value<-jsonRespParsed$value
df<-as.data.frame(value%>%bind_rows%>%select(name,url,description))
df

name,url,description
<chr>,<chr>,<chr>
Category: Coronavirus and Education,https://hechingerreport.org/special-reports/coronavirus/,"When the coronavirus pandemic first struck and classes shifted online last spring, Sophia Joffe was in 11th grade. Her private school in Toronto, Ontario, had made the transition admirably well, she thought, but she wondered what online tools existed to ..."
Coronavirus | Vancouver Sun,https://vancouversun.com/tag/coronavirus/coronavirus/coronavirus/coronavirus/?more=coronavirus&from=100,"COVID-19 update for Sept. 23: Here's the latest on coronavirus in B.C. Here's your daily update with everything you need to know on the novel coronavirus situation in B.C. for Sept. 23, 2020. B.C ..."
Coronavirus | National Post,https://nationalpost.com/tag/coronavirus/?from=25,Donald Trump calls distinguished COVID-19 expert Anthony Fauci a 'disaster' Trump aired his frustration with the scientist during a call meant to reassure campaign staff he still has a path to ...
Coronavirus | The Star Phoenix,https://thestarphoenix.com/tag/coronavirus/coronavirus/?from=25,"The southeastern city of Yorkton is reporting COVID-19 diagnoses at its hospital, a local gym and its RCMP detachment. The former provincial politician says more needs to be done to stimulate the ..."
Coronavirus | Ottawa Citizen,https://ottawacitizen.com/tag/coronavirus/?from=25,WHO: Letting virus spread through herd immunity is 'scientifically and ethically problematic' Imagine the moral implications of allowing large parts of the population to become ill to get immunity ...
Coronavirus | The Province,https://theprovince.com/tag/coronavirus/?from=25,"“Everyone wants the best for their pets if they have to leave them during a vacation,"" says owner Tatiana Custode. A panel at the UBCM convention brought together large and small municipalities ..."
Coronavirus | Toronto Sun,https://torontosun.com/tag/coronavirus/?from=50,"Ontario hospitals are calling on the Doug Ford government to return all of the GTA and Ottawa regions to Stage 2. Hidden in the donated tents, among the bikes, tarps, coolers, signs, drug ..."
Coronavirus | Montreal Gazette,https://montrealgazette.com/tag/coronavirus/?from=25,"Three latest cases were recorded at 12090 Sherbrooke St. E. and 1275 Rosemont Blvd. in Montreal, and at 1400 Moody Blvd. in Terrebonne. Italy was the first country in Europe to be slammed by COVID ..."
Coronavirus | Calgary Herald,https://calgaryherald.com/en/tag/tag/coronavirus/tag/coronavirus/?more=coronavirus&from=75,"Ottawa cannot afford to follow through on its throne speech promises without Alberta’s energy sector, Premier Jason Kenney warned Thursday, taking another swipe at the federal government’s ..."


In [ ]:
library("datasets")
library("aws.s3")
download_url<-function(name, url){
  html<-read_html(url)
  article<-html %>%
    html_nodes('p') %>%
    html_text()
  s_v	<- get_sentences(article)
  datalist = list()
  for (i in 1:length(s_v)){
    szu_score <- get_sentiment(s_v[i],	method="syuzhet")
    afinn_score <- get_sentiment(s_v[i],	method="afinn")
    bing_score <- get_sentiment(s_v[i],	method="bing")
    nrc_score <- get_sentiment(s_v[i],	method="nrc")
    dat <- data.frame("title"=s_v[i],"szu_score"=szu_score,"afinn_score"=afinn_score,"bing_score"=bing_score,"nrc_score"=nrc_score)
    # print(dat)
    dat$url <- url  # maybe you want to keep track of which iteration produced it?
    datalist[[i]] <- dat # add it to your list
  }
  big_data = do.call(rbind, datalist)
  
  
  write.table(big_data, "myDF.csv", sep = ",", col.names = !file.exists("myDF.csv"), append = T)

  
  
}
a<-mapply(download_url, df$name, df$url)
aws.s3::put_object(file = "myDF.csv",
    bucket = "navdeep-assignment", object = "sentiment_analysis.csv")
# if(require("utils")){
#   s3write_using(big_data, FUN = write.csv,
#                   bucket = "navdeep-assignment",
#                   object = paste("sentiments_", name, sep=""))
# }


[1] TRUE

In [ ]:
sentiment_df <- read.csv(file = 'myDF.csv')
sentiment_df

ERROR: ignored